In [23]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
dataset_dir = 'Dataset/generated_images'
original_dir = os.path.join(dataset_dir, 'original_350x500')
resized_dir = os.path.join(dataset_dir, 'resized_224x224')
characters_dir = os.path.join(dataset_dir, 'characters')

In [25]:
waldo = cv2.imread(os.path.join(characters_dir, 'waldo.png'))
wilma = cv2.imread(os.path.join(characters_dir, 'wilma.png'))

libpng warning: iCCP: known incorrect sRGB profile


In [26]:
waldo = cv2.cvtColor(waldo, cv2.COLOR_BGR2RGB)
wilma = cv2.cvtColor(wilma, cv2.COLOR_BGR2RGB)
waldo = cv2.resize(waldo, (224, 224))
wilma = cv2.resize(wilma, (224, 224))
waldo = preprocess_input(waldo)
wilma = preprocess_input(wilma)

In [28]:
X = []
y = []
for i in range(5000):
    # Read in original image
    img_orig = cv2.imread(os.path.join(original_dir, f'image_{i}.png'))
    # Resize original image
    img_resized = cv2.imread(os.path.join(resized_dir, f'resized_image_{i}.png'))
    # Preprocess resized image
    img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_resized = preprocess_input(img_resized)
    # Check if Waldo is present in the image
    if np.sum(cv2.matchTemplate(img_orig.astype(np.float32), waldo.astype(np.float32), cv2.TM_CCOEFF_NORMED)) > 0.8:
        X.append(img_resized)
        y.append(0) # Waldo label
    # Check if Wilma is present in the image
    elif np.sum(cv2.matchTemplate(img_orig.astype(np.float32), wilma.astype(np.float32), cv2.TM_CCOEFF_NORMED)) > 0.8:
        X.append(img_resized)
        y.append(1) # Wilma label


In [29]:
X = np.array(X)
y = np.array(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [31]:
for layer in base_model.layers:
    layer.trainable = False

In [32]:
x = base_model.output
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

In [34]:
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stop])

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)

print(f'Validation loss: {loss:.4f}')
print(f'Validation accuracy: {accuracy:.4f}')

# Save the model
model.save('waldo_wilma_model.h5')

: 

: 